In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import spacy

train_df = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
test_df = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")


In [3]:
train_df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
test_df.head()


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [5]:
count_vectorizer = feature_extraction.text.CountVectorizer()

example_train_vectors = count_vectorizer.fit_transform(train_df["comment_text"][0:5])


In [6]:
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())


(1, 157)
[[0 0 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0
  1 0 0 0 0 1 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0
  0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 3 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1
  1 0 0 1 1 0 1 0 0 0 1 0 0]]


In [7]:
train_vectors = count_vectorizer.fit_transform(train_df["comment_text"])

test_vectors = count_vectorizer.transform(test_df["comment_text"])


In [8]:
clf = MultinomialNB()


In [9]:
scores_toxic = model_selection.cross_val_score(clf, train_vectors, train_df["toxic"], cv=3, scoring="roc_auc")
scores_severeToxic = model_selection.cross_val_score(clf, train_vectors, train_df["severe_toxic"], cv=3, scoring="roc_auc")
scores_obscene = model_selection.cross_val_score(clf, train_vectors, train_df["obscene"], cv=3, scoring="roc_auc")
scores_threat = model_selection.cross_val_score(clf, train_vectors, train_df["threat"], cv=3, scoring="roc_auc")
scores_insult = model_selection.cross_val_score(clf, train_vectors, train_df["insult"], cv=3, scoring="roc_auc")
scores_identityHate = model_selection.cross_val_score(clf, train_vectors, train_df["identity_hate"], cv=3, scoring="roc_auc")

print(scores_toxic)
print(scores_severeToxic)
print(scores_obscene)
print(scores_threat)
print(scores_insult)
print(scores_identityHate)


[0.89101933 0.88135612 0.88419258]
[0.90375814 0.86725322 0.87379962]
[0.88007343 0.88259693 0.88055262]
[0.735473   0.76309395 0.73346823]
[0.88153647 0.87118281 0.8675816 ]
[0.78658717 0.76953167 0.78100333]


In [10]:
def getProbabilities(predicts):
    probabilities = []
    for probability in predicts:
        probabilities.append(probability[1])

    return probabilities

clf.fit(train_vectors, train_df["toxic"])
probsToxic = clf.predict_proba(test_vectors) 
predToxic = pd.Series(getProbabilities(probsToxic)).apply(lambda x: float(x))

clf.fit(train_vectors, train_df["severe_toxic"])
probsSevereToxic = clf.predict_proba(test_vectors)
predSevereToxic = pd.Series(getProbabilities(probsSevereToxic)).apply(lambda x: float(x))

clf.fit(train_vectors, train_df["obscene"])
probsObscene = clf.predict_proba(test_vectors)
predObscene = pd.Series(getProbabilities(probsObscene)).apply(lambda x: float(x))

clf.fit(train_vectors, train_df["threat"])
probsThreat = clf.predict_proba(test_vectors)
predThreat = pd.Series(getProbabilities(probsThreat)).apply(lambda x: float(x))

clf.fit(train_vectors, train_df["insult"])
probsInsult = clf.predict_proba(test_vectors)
predInsult = pd.Series(getProbabilities(probsInsult)).apply(lambda x: float(x))

clf.fit(train_vectors, train_df["identity_hate"])
probsIdentityHate = clf.predict_proba(test_vectors)
predIdentityHate = pd.Series(getProbabilities(probsIdentityHate)).apply(lambda x: float(x))


In [11]:
result = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

result['toxic'] = predToxic
result['severe_toxic'] = predSevereToxic
result['obscene'] = predObscene
result['threat'] = predThreat
result['insult'] = predInsult
result['identity_hate'] = predIdentityHate

result.to_csv('submission.csv', index=False)
